In [71]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Hyperparameters

In [72]:
COLUMNS_TO_KEEP = ["direct_rad:W", "clear_sky_rad:W", 'diffuse_rad:W', 'direct_rad_1h:J','pv_measurement']
LEARNING_RATE = 0.001
BATCH_SIZE = 10
NUM_FEATURES = len(COLUMNS_TO_KEEP) - 1 # -1 because pv_measurement is the target
FEATURE_SIZE = 4 # 7 days of hourly data

# Neural net

In [73]:
class SolarPredictionNet(nn.Module):
    def __init__(self, input_size, num_channels):
        super(SolarPredictionNet, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv1d(num_channels, 32, kernel_size=1, stride=1, padding=0)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=1, stride=1, padding=0)
        #self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)

        # Compute the output size after convolutions and pooling
        out_size = input_size // 4  # Divided by 4 because of two pooling layers

        # Fully connected layers
        self.fc1 = nn.Linear(out_size * 64, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        # x = self.pool(x)
        x = self.relu(self.conv2(x))
        # x = self.pool(x)

        # Flatten
        x = x.view(x.size(0), -1)
        
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

# Load dataset

In [74]:
# Load data from Parquet files
df_data = pd.read_parquet('data/A/X_train_observed.parquet')
df_target = pd.read_parquet('data/A/train_targets.parquet')

# Merge the datasets
df_merged = pd.merge(df_data, df_target, left_on='date_forecast', right_on='time', how='inner')
df_merged = df_merged[COLUMNS_TO_KEEP]

# Separate the features and targets
y = df_merged['pv_measurement']
X = df_merged.drop('pv_measurement', axis=1) # Replace 'target_column_name' with the name of your target column

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X.values, dtype=torch.float32).unsqueeze(2)  # [batch_size, channels, 1]
y_tensor = torch.tensor(y.values, dtype=torch.float32).unsqueeze(1)  # Convert y to a 2D tensor

# Create a custom dataset
class SolarDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create data loaders
train_dataset = SolarDataset(X_tensor, y_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# ... [Neural Network and Training code from previous messages]


# Training Loop

In [75]:
def train_model(model, train_loader):
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    for epoch in range(NUM_EPOCHS):
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch + 1}/{NUM_EPOCHS}, Loss: {loss.item()}")

    print("Training complete!")

In [76]:
model = SolarPredictionNet(FEATURE_SIZE, NUM_FEATURES)
train_model(model, train_loader)

Epoch 1/100, Loss: 680.6752319335938
Epoch 2/100, Loss: 426.3002014160156
Epoch 3/100, Loss: 349.064697265625
Epoch 4/100, Loss: 217.43714904785156
Epoch 5/100, Loss: 1009.0840454101562
Epoch 6/100, Loss: 50.46498489379883
Epoch 7/100, Loss: 21.820398330688477
Epoch 8/100, Loss: 584.3775024414062
Epoch 9/100, Loss: 315.2033386230469
Epoch 10/100, Loss: 271.19195556640625
Epoch 11/100, Loss: 185.15980529785156
Epoch 12/100, Loss: 206.5850067138672
Epoch 13/100, Loss: 693.9456176757812
Epoch 14/100, Loss: 106.92974853515625
Epoch 15/100, Loss: 170.02127075195312
Epoch 16/100, Loss: 302.4627990722656
Epoch 17/100, Loss: 219.7935791015625
Epoch 18/100, Loss: 392.33843994140625
Epoch 19/100, Loss: 174.2693328857422
Epoch 20/100, Loss: 310.2723693847656
Epoch 21/100, Loss: 148.2890625
Epoch 22/100, Loss: 181.5765838623047
Epoch 23/100, Loss: 124.07129669189453
Epoch 24/100, Loss: 189.64785766601562
Epoch 25/100, Loss: 161.993408203125
Epoch 26/100, Loss: 69.2517318725586
Epoch 27/100, Loss: 